## Final Report: Deep Dream
CS 344  
Ridge DeJong




### *Vision*
This project revolves around Deep Dream, which is in the category of image processing. It was originally created by Google, and its purpose was to help get a better understanding of how deep neural networks see images. Deep dream works by reversing the neural network so that instead of asking the model to identify an object in an image, you identify an object and tell the model to find that object in the image. If you do this to an image that doesn't contain the identified object, the model will still try to find that object because you said it is there. This will cause the model to start recreating the image in a way that it creates the illusion that the object is there. The significance of this is that when a model makes these changes, we can visually see the small pieces that the model is looking for, indicating what the model knows (and doesn't know) about that object and what direction the model needs to go to become more accurate. Therefore, the purpose of this project was open up the hood of Deep Dream to better understand how it works and what its capabilities are. 

### *Background*

This work was based off the Deep Dream tutorial by Chollet (https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/8.2-deep-dream.ipynb). Similar to the tutorial, this project used a pre-trained convolutional neural network called Inception_v3 as the model. A CNN was chosen above other neural networks because this type of network excels at  processing images efficiently and accurately. This CNN was trained on ImageNet, which is a database composed of more than a million hand-labeled images. Other technologies involved in this program were activation and loss of different layers (https://github.com/kvlinden-courses/cs344-code/blob/master/u08features/backpropagation.ipynb), along with a gradient ascent process (https://developers.google.com/machine-learning/glossary#gradient-descent) which have all been seen before in class (we studied gradient descent but gradient ascent is the same thing, just maximizing loss instead of minimizing). Some new technologies were preprocessing and deprocessing images, as well as detail injection into images. The point of preprocessing an image is to prepare it for the model so that it is easier to analyze. In this case, preprocessing meant resizing the image and formatting it into an appropriate tensor. Deprocessing is of course the opposite of this. After the model has finished analyzing the tensor, the tensor is converted and resized back into an image so that the results can be seen. Detail injection occurs between octaves where the image is upscaled. Since a smaller image would lose detail and become blurry when scaled bigger, details from the original image are injected to counteract this. It works by calculating the difference between the original image resized to the smaller image and the original image resized to the upscaled image. This difference therefore quantifies the details lost when going from the smaller image to the upscaled image, which can then be injected. The heart of this Deep Dream algorithm is shown below, while the full code can be found in the repo under "Deep Dream Code".

In [0]:
# Fill this to the path to the image you want to use
base_image_path = '/dog.jpg'

# Load the image into a Numpy array
img = preprocess_image(base_image_path)

# We prepare a list of shape tuples
# defining the different scales at which we will run gradient ascent
original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)

# Reverse list of shapes, so that they are in increasing order
successive_shapes = successive_shapes[::-1]

# Resize the Numpy array of the image to our smallest scale
original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
    print('Processing image shape', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img,
                          iterations=iterations,
                          step=step,
                          max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img

    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)
    save_img(img, fname='dream_at_scale_' + str(shape) + '.png')

save_img(img, fname='final_dream.png')

### *Implementation*

The project started by copying the tutorial mentioned above and getting it to run. Then, different types of images were fed to the model to see what the Deep Dream program did to them. There was a colorful image, a blank image, a static image, a high resolution image, a similar low resolution image, images with animals, a cartoon animal image, a clear image with lots of people, and a blurry image of people. With all these images, the first modification was to adjust some or all of the 11 layer activation coefficients labeled mixed0 through mixed 10. These coefficients specified how much of each layer's activation contributed to the loss being maximixed. The chunk of code where this was done is shown below.


In [0]:
layer_contributions = {
    'mixed0': 0.001,
    'mixed1': 0.05,
    'mixed2': 0.5,
    'mixed3': 1,
    'mixed4': 1,
    'mixed5': 0.5,
    'mixed6': 0.1,
    'mixed7': 0.05,
    'mixed8': 0.005,
    'mixed9': 0.0001,
    'mixed10': 0.0001,
}

Second, for all these images the hyperparameters were adjusted. These included 'step' which was the gradient ascent step size, 'num_octave' which was the number of scales that the gradient ascent was run for, 'octave_scale' which was the size ratio between these different scales, 'iterations' which was the number of ascent steps for each scale, and lastly 'max_loss' which was the limit at which the gradient ascent process would terminate if reached. By using many different combinations of these hyperparameters over a diverse group of images, a good understanding of Deep Dream was gained, which is explained in the Results section. The code below shows the hyperparameters that were set.

In [0]:
step = 0.05  # Gradient ascent step size
num_octave = 5  # Number of scales at which to run gradient ascent
octave_scale = 1.6  # Size ratio between scales
iterations = 20  # Number of ascent steps per scale

# If our loss gets larger than 10,
# we will interrupt the gradient ascent process, to avoid ugly artifacts
max_loss = 60.

lower level = geometric pattern

In [3]:
%%html
<img src="moon3.jpg" width="800" />